In [14]:
import numpy as np
import pandas as pd
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [8]:
X = pd.read_parquet('../../data/preprocessed/test_x.pq')
y = pd.read_parquet('../../data/preprocessed/test_y.pq')

In [9]:
# Open the file containing the pickled model in binary mode
with open('../../models/linreg.pkl', 'rb') as file:
    # Load the model using pickle
    model = pickle.load(file)

# Close the file
file.close()

# Now 'model' contains the loaded model


In [54]:
def evaluation(X_test, y_test, y_pred, pca=False, examples=True):
    list1 = [item for sublist in y_test.values for item in sublist]
    list2 = [item for sublist in y_pred for item in sublist]


    
    

    # Calculate absolute differences element-wise
    abweichung = []
    for a, b in zip(list1, list2):
        
        abweichung.append(abs(a - b))    # Give points
    points = []
    for diff in abweichung:
        if diff < 0.05:
            points.append(1.0)
        elif 0.05 <= diff < 0.1:
            points.append(0.5)
        elif 0.1 <= diff < 0.5:
            points.append(0.25)
        else:
            points.append(0) 

    list1 = points

    points2 = [] 
    consecutive_count =0      
    for i in range(0, len(list1)):
        
        if list1[i] >=0.5 and consecutive_count == 0 :

            points2.append (1)

        elif list1[i] >=0.5 and consecutive_count != 0:

            if consecutive_count == 1:
                points2.append (.5)
                points2.append(1)
     
            elif 2 <= consecutive_count <= 10:
                points2.extend([0.25] * consecutive_count)
                points2.append(1)
            else:
                points2.extend([0] * consecutive_count)
                points2.append(1)

            consecutive_count = 0# Reset consecutive count

        else:
            consecutive_count+=1

    
    if consecutive_count == 1:
        points2.append (.5)
    elif 2 <= consecutive_count <= 10:
        points2.extend([0.25] * consecutive_count)
    else:
        points2.extend([0] * consecutive_count)



    total = sum(points)+sum(points2) 
    maxpoint = len(points)*2

    print(" Punkte aus abweichungslist:", sum(points), "Punkte aus Zeitabweichung:", sum(points2), "max Punkte:",len(points))

    print(total/maxpoint)



 

    if not pca and examples:
        # Print some example predictions
        example_indices = [0, 1, 2, 3, 4]
        for i in example_indices:
            print(f"{X_test.index[i]}: | Actual y: {y_test.iloc[i, 0]} | Predicted y: {y_pred[i]}")


 
        
    
    
    





# Utility Functions
def add_time_features(df):
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    daytime_arr = []
    for idx in df.index:
        if df.loc[idx, "hour"] >= 6 and df.loc[idx, "hour"] <= 18:
            daytime_arr.append(1)
        else:
            daytime_arr.append(0)

    df['day_time'] = daytime_arr
    df['night_time'] = 1 - df['day_time']
    return df

In [55]:

X = add_time_features(X)

# Make predictions on the testing data
y_pred = model.predict(X)

evaluation(X, y, y_pred, examples=False)


 Punkte aus abweichungslist: 106085.75 Punkte aus Zeitabweichung: 112232.5 max Punkte: 205203
0.5319567696378708
